In [1]:
import numpy as np
import ctypes
import matplotlib.pyplot as plt
import popeye.og as og
import popeye.utilities as utils
from popeye.visual_stimulus import VisualStimulus, simulate_bar_stimulus
from popeye import css
from scipy.io import loadmat
import time
import nibabel as nib
from nilearn import plotting
import os
import tqdm
import pickle
import multiprocessing as mp
import time
from ipywidgets import interact, widgets

# Load helper functions
from dataloader import set_paths, load_stimuli, copy_files
%load_ext autoreload
%autoreload 2

In [2]:
# Initialize parameters
params = {}
params['subjID'] = 'JC'
# Got these from Zhengang, and he got it from rsvp_params.txt
params['viewingDistance'] = 83.5 # in cm
params['screenWidth'] = 36.2 # in cm
params['scaleFactor'] = 0.5
params['tr_length'] = 1.3 # in seconds (got this from Aditya, need to check)
params['dtype'] = ctypes.c_int16

p = set_paths(params)

In [3]:
bar, stim_params = load_stimuli(p)
bar = bar[:, :, 0:201] # Additional few slices with empty frames to match the new scan sequence, can be ignored for subject JC
copy_files(p, params)

# Extract number of TRs
func_data = nib.load(p['pRF_func'])
params['nTRs'] = func_data.shape[-1]

Subject folder already exists


In [4]:
# create stimulus object from popeye
stimulus = VisualStimulus(bar,
                          params['viewingDistance'],
                          params['screenWidth'],
                          params['scaleFactor'],
                          params['tr_length'],
                          params['dtype'])

In [5]:
# model to fit to
method = 'ss5'
scan_data = nib.load(p['pRF_func']).get_fdata()
brainmask_data = nib.load(p['pRF_brainmask']).get_fdata() != 0
# Resample brainmask if first 2 dimensions are twice the third dimension
if brainmask_data.shape[0] == 2*brainmask_data.shape[2]:
    brainmask_data = brainmask_data[::2, ::2, :]

In [9]:
# Testing only on visual ROIs
# Load visual ROIs
lh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V1.nii.gz')).get_fdata()
lh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V2d.nii.gz')).get_fdata()
lh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3d.nii.gz')).get_fdata()
lh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'lh.V3AB.nii.gz')).get_fdata()
rh_v1 = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V1.nii.gz')).get_fdata()
rh_v2d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V2d.nii.gz')).get_fdata()
rh_v3d = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3d.nii.gz')).get_fdata()
rh_v3ab = nib.load(os.path.join(p['pRF_data'], params['subjID'], 'roi_mdd', 'rh.V3AB.nii.gz')).get_fdata()
# Combine all ROIs using boolean OR
visual_rois = lh_v1 + lh_v2d + lh_v3d + lh_v3ab + rh_v1 + rh_v2d + rh_v3d + rh_v3ab
visual_rois = visual_rois > 0
visual_rois = lh_v1 #+ rh_v2d
visual_rois = visual_rois > 0

In [8]:
css_model = css.CompressiveSpatialSummationModel(stimulus, utils.spm_hrf)
css_model.hrf_delay = 0
css_model.mask_size = 1

# Create scan data just for visual ROIs
scan_data_visual = scan_data.copy()
scan_data_visual[~visual_rois] = 0

[xi, yi, zi] = np.nonzero(visual_rois)
indices = [(xi[i], yi[i], zi[i]) for i in range(len(xi))]

# set search grid
x_grid = (-5, 5)
y_grid = (-5, 5)
s_grid = (0.25, 2.0)
n_grid = (0.05, 0.5)
grids = (x_grid, y_grid, s_grid, n_grid,)

# set search bounds
x_bounds = (-10, 10)
y_bounds = (-10, 10)
sigma_bounds = (1/css_model.stimulus.ppd, 5.25)
n_bounds = (1e-8, 1)
beta_bounds = (1e-8, None)
baseline_bounds = (None, None)
bounds = (x_bounds, y_bounds, sigma_bounds, n_bounds, beta_bounds, baseline_bounds)

In [11]:
# Initialize an array to store model fits
rf_fit = np.empty((scan_data.shape[0], scan_data.shape[1], scan_data.shape[2], 8))
# pol ve ecc sigmamajor exponent x0 y0 b

In [9]:
for ix in range(scan_data.shape[0]):
    for iy in range(scan_data.shape[1]):
        for iz in range(scan_data.shape[2]):
            # Fit data only for voxels in visual ROIs
            if visual_rois[ix, iy, iz] == 1:
                print(f"Voxel: ({ix}, {iy}, {iz})")
                voxel_data = scan_data[ix, iy, iz, :]
                # voxel_data = voxel_data.reshape((1, -1))
                # print(voxel_data.shape)
                print(f"Length of voxel data: {len(voxel_data)}")
                fit = css.CompressiveSpatialSummationFit(css_model,
                                                   voxel_data,
                                                   grids,
                                                   bounds,
                                                   Ns=3,
                                                   voxel_index=(1, 2, 3),
                                                   auto_fit=True,
                                                   verbose=2)
                # rf_fit[ix, iy, iz, 0] = fit.theta
                # rf_fit[ix, iy, iz, 1] = fit.3
                # rf_fit[ix, iy, iz, 5] = fit.estimate[0]
                # rf_fit[ix, iy, iz, 6] = fit.estimate[1]
                # rf_fit[ix, iy, iz, 3] = fit.estimate[2]
                # rf_fit[ix, iy, iz, 4] = fit.estimate[3]
                print(fit)

Voxel: (49, 99, 61)
Length of voxel data: 201
The gridfit solution was [   -1.89473684    -1.89473684     3.31882348     0.1
   269.21389965 16115.94332537], starting gradient descent ...
VOXEL=(001,002,003)   TIMEMS=545433   RSQ=0.88  EST=[    3.927      2.6796    10.495      0.1      269.2139 16115.9433]
Voxel: (49, 99, 62)
Length of voxel data: 201


KeyboardInterrupt: 